# Trabajo Práctico 4
Para las imágenes suministradas:
1. Implementar un extractor de características LBP básico (sin uniformidad, 8 vecinos)
2. Realizar el histograma de características LBP de la imagen
3. Comparar los histogramas

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
cv.destroyAllWindows()

In [2]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib

Using matplotlib backend: MacOSX


In [3]:
cam = cv.VideoCapture(1)

In [4]:
ret, frame = cam.read()
print("Frame shape:", frame.shape)
cv.imshow('Frame', frame)
cam.release()

Frame shape: (720, 1280, 3)


In [5]:
roi = None
roi_pts = [(0,0), (0,0)]
cropping = False

def select_roi(event, x, y, flags, param):
    global frame, roi, roi_pts, cropping
    
    if event == cv.EVENT_LBUTTONDOWN:
        print(event)
        print(x, y)
        roi_pts[0] = (x, y)
        cropping = True

    elif event == cv.EVENT_LBUTTONUP:
        print(event)
        print(x, y)
        
        roi_pts[1] = (x, y)
        cropping = False

        frame_cp = frame.copy()
        cv.rectangle(frame_cp, roi_pts[0], roi_pts[1], (0, 255, 0), 2)
        cv.imshow('Frame', frame_cp)

        yi = min(roi_pts[0][0], roi_pts[1][0])
        xi = min(roi_pts[0][1], roi_pts[1][1])
        yf = max(roi_pts[0][0], roi_pts[1][0])
        xf = max(roi_pts[0][1], roi_pts[1][1])
        
        roi = frame[xi:xf, yi:yf]
        
        if roi is not None:
            print("ROI correctly selected")
            cv.imshow('ROI', roi)
        else:
            print("Error")

cv.setMouseCallback('Frame', select_roi)

print("Click & Drag to select ROI - Hit any key to continue")
cv.waitKey(0)

Click & Drag to select ROI - Hit any key to continue
1
841 406
4
1041 577
ROI correctly selected


113

In [6]:
if roi is not None:
    print("ROI correctly selected")
else:
    print("Error")

ROI correctly selected


In [7]:
print(roi_pts[0][0], roi_pts[0][1])
print(roi_pts[1][0], roi_pts[1][1])

yi = min(roi_pts[0][0], roi_pts[1][0])
xi = min(roi_pts[0][1], roi_pts[1][1])
yf = max(roi_pts[0][0], roi_pts[1][0])
xf = max(roi_pts[0][1], roi_pts[1][1])

print(xi,xf,yi,yf)

841 406
1041 577
406 577 841 1041


In [15]:
# %matplotlib inline
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)

fig, ax = plt.subplots(1,1)

major_ticks_top=np.linspace(0,255,16)
minor_ticks_top=np.linspace(0,255,128)

ax.set_xticks(major_ticks_top)
ax.set_xticks(minor_ticks_top, minor=True)

ax.hist(hsv_roi[:,:,0].ravel(), bins=30, label="H")
ax.hist(hsv_roi[:,:,1].ravel(), bins=30, label="S")
ax.hist(hsv_roi[:,:,2].ravel(), bins=30, label="V")
ax.legend()
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.grid()

fig.show()

In [29]:
mask = cv.inRange(hsv_roi, np.array((75., 40., 6.)), np.array((110., 204., 68.)))
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

plt.figure()
plt.subplot(121)
plt.imshow(cv.cvtColor(roi, cv.COLOR_BGR2RGB))
plt.subplot(122)
plt.imshow(mask)

In [30]:
x = xi
y = yi
w = xf - xi
h = yf - yi
track_window = (x, y, w, h)
print(track_window)

# Condición de terminación: 10 iteraciones o moverse al menos 1pt
#----------------------------------------------------------------
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10 )

camera = cv.VideoCapture(1)

while(1):
    ret, frame = camera.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        dst = cv.medianBlur(dst,9)
        
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        
        # Dibujamos la ventana sobre la imagen
        #=====================================
        
        # Con escalamiento y rotación de la ventana de seguimiento
        #---------------------------------------------------------
        #pts = cv.boxPoints(ret)                      # Encuentra los 4 vértices del rectángulo rotado
        #pts = np.int0(pts)                           # int0 (entero usado para indexado)
        #img2 = cv.polylines(frame,[pts],True, 255,2) # Dibujo del rectángulo
        
        # Con escalamiento de ventana - Sin rotación
        #-------------------------------------------
        x,y,w,h = track_window
        img2 = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        
        cv.imshow('Seguimiento',img2)
        cv.imshow('Mask',dst)
        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

camera.release()

(406, 841, 171, 200)


In [45]:
camera.release()
cv.destroyAllWindows()